# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

---

## <font face="仿宋">第四部分导读

&emsp;&emsp;<font face="仿宋">在案例的第二、三部分中，我们详细介绍了关于特征工程的各项技术，特征工程技术按照大类来分可以分为数据预处理、特征衍生、特征筛选三部分，其中特征预处理的目的是为了将数据集整理、清洗到可以建模的程度，具体技术包括缺失值处理、异常值处理、数据重编码等，是建模之前必须对数据进行的处理和操作；而特征衍生和特征筛选则更像是一类优化手段，能够帮助模型突破当前数据集建模的效果上界。并且我们在第二部分完整详细的介绍机器学习可解释性模型的训练、优化和解释方法，也就是逻辑回归和决策树模型。并且此前我们也一直以这两种算法为主，来进行各个部分的模型测试。

&emsp;&emsp;<font face="仿宋">而第四部分，我们将开始介绍集成学习的训练和优化的实战技巧，尽管从可解释性角度来说，集成学习的可解释性并不如逻辑回归和决策树，但在大多数建模场景下，集成学习都将获得一个更好的预测结果，这也是目前效果优先的建模场景下最常使用的算法。

&emsp;&emsp;<font face="仿宋">总的来说，本部分内容只有一个目标，那就是借助各类优化方法，抵达每个主流集成学习的效果上界。换而言之，本部分我们将围绕单模优化策略展开详细的探讨，涉及到的具体集成学习包括随机森林、XGBoost、LightGBM、和CatBoost等目前最主流的集成学习算法，而具体的优化策略则包括超参数优化器的使用、特征衍生和筛选方法的使用、单模型自融合方法的使用，这些优化方法也是截至目前，提升单模效果最前沿、最有效、同时也是最复杂的方法。其中有很多较为艰深的理论，也有很多是经验之谈，但无论如何，我们希望能够围绕当前数据集，让每个集成学习算法优化到极限。值得注意的是，在这个过程中，我们会将此前介绍的特征衍生和特征筛选视作是一种模型优化方法，衍生和筛选的效果，一律以模型的最终结果来进行评定。而围绕集成学习进行海量特征衍生和筛选，也才是特征衍生和筛选技术能发挥巨大价值的主战场。

&emsp;&emsp;<font face="仿宋">而在抵达了单模的极限后，我们就会进入到下一阶段，也就是模型融合阶段。需要知道的是，只有单模的效果到达了极限，进一步的多模型融合、甚至多层融合，才是有意义的，才是有效果的。

---

# <center>Part 4.集成算法的训练与优化技巧

In [2]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings
warnings.filterwarnings('ignore')

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

# 自定义模块
from telcoFunc import *

# 导入特征衍生模块
import features_creation as fc
from features_creation import *

# 导入模型融合模块
import manual_ensemble as me
from manual_ensemble import *

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc

&emsp;&emsp;然后执行Part 1中的数据清洗相关工作：

In [2]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [3]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

&emsp;&emsp;同时，创建自然编码后的数据集以及经过时序特征衍生的数据集：

In [4]:
# 划分训练集和测试集
train, test = train_test_split(tcc, random_state=22)

X_train = train.drop(columns=[ID_col, target]).copy()
X_test = test.drop(columns=[ID_col, target]).copy()

y_train = train['Churn'].copy()
y_test = test['Churn'].copy()

X_train_seq = pd.DataFrame()
X_test_seq = pd.DataFrame()

# 年份衍生
X_train_seq['tenure_year'] = ((72 - X_train['tenure']) // 12) + 2014
X_test_seq['tenure_year'] = ((72 - X_test['tenure']) // 12) + 2014

# 月份衍生
X_train_seq['tenure_month'] = (72 - X_train['tenure']) % 12 + 1
X_test_seq['tenure_month'] = (72 - X_test['tenure']) % 12 + 1

# 季度衍生
X_train_seq['tenure_quarter'] = ((X_train_seq['tenure_month']-1) // 3) + 1
X_test_seq['tenure_quarter'] = ((X_test_seq['tenure_month']-1) // 3) + 1

# 独热编码
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_seq)

seq_new = list(X_train_seq.columns)

# 创建带有列名称的独热编码之后的df
X_train_seq = pd.DataFrame(enc.transform(X_train_seq).toarray(), 
                           columns = cate_colName(enc, seq_new, drop=None))

X_test_seq = pd.DataFrame(enc.transform(X_test_seq).toarray(), 
                          columns = cate_colName(enc, seq_new, drop=None))

# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

In [5]:
ord_enc = OrdinalEncoder()
ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(ord_enc.transform(X_train[category_cols]), columns=category_cols)
X_train_OE.index = X_train.index
X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(ord_enc.transform(X_test[category_cols]), columns=category_cols)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

然后是模型融合部分所需的第三方库、准备的数据以及训练好的模型：

In [6]:
# 实例化KFold评估器
kf = KFold(n_splits=5, random_state=12, shuffle=True)

# 重置训练集和测试集的index
X_train_OE = X_train_OE.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

train_part_index_l = []
eval_index_l = []

for train_part_index, eval_index in kf.split(X_train_OE, y_train):
    train_part_index_l.append(train_part_index)
    eval_index_l.append(eval_index)
    
# 训练集特征
X_train1 = X_train_OE.loc[train_part_index_l[0]]
X_train2 = X_train_OE.loc[train_part_index_l[1]]
X_train3 = X_train_OE.loc[train_part_index_l[2]]
X_train4 = X_train_OE.loc[train_part_index_l[3]]
X_train5 = X_train_OE.loc[train_part_index_l[4]]

# 验证集特征
X_eval1 = X_train_OE.loc[eval_index_l[0]]
X_eval2 = X_train_OE.loc[eval_index_l[1]]
X_eval3 = X_train_OE.loc[eval_index_l[2]]
X_eval4 = X_train_OE.loc[eval_index_l[3]]
X_eval5 = X_train_OE.loc[eval_index_l[4]]

# 训练集标签
y_train1 = y_train.loc[train_part_index_l[0]]
y_train2 = y_train.loc[train_part_index_l[1]]
y_train3 = y_train.loc[train_part_index_l[2]]
y_train4 = y_train.loc[train_part_index_l[3]]
y_train5 = y_train.loc[train_part_index_l[4]]

# 验证集标签
y_eval1 = y_train.loc[eval_index_l[0]]
y_eval2 = y_train.loc[eval_index_l[1]]
y_eval3 = y_train.loc[eval_index_l[2]]
y_eval4 = y_train.loc[eval_index_l[3]]
y_eval5 = y_train.loc[eval_index_l[4]]

train_set = [(X_train1, y_train1), 
             (X_train2, y_train2), 
             (X_train3, y_train3), 
             (X_train4, y_train4), 
             (X_train5, y_train5)]

eval_set = [(X_eval1, y_eval1), 
            (X_eval2, y_eval2), 
            (X_eval3, y_eval3), 
            (X_eval4, y_eval4), 
            (X_eval5, y_eval5)]

In [7]:
# 随机森林模型组
grid_RF_1 = load('./models/grid_RF_1.joblib') 
grid_RF_2 = load('./models/grid_RF_2.joblib') 
grid_RF_3 = load('./models/grid_RF_3.joblib') 
grid_RF_4 = load('./models/grid_RF_4.joblib') 
grid_RF_5 = load('./models/grid_RF_5.joblib') 

RF_1 = grid_RF_1.best_estimator_
RF_2 = grid_RF_2.best_estimator_
RF_3 = grid_RF_3.best_estimator_
RF_4 = grid_RF_4.best_estimator_
RF_5 = grid_RF_5.best_estimator_

RF_l = [RF_1, RF_2, RF_3, RF_4, RF_5]

# 决策树模型组
grid_tree_1 = load('./models/grid_tree_1.joblib')
grid_tree_2 = load('./models/grid_tree_2.joblib')
grid_tree_3 = load('./models/grid_tree_3.joblib')
grid_tree_4 = load('./models/grid_tree_4.joblib')
grid_tree_5 = load('./models/grid_tree_5.joblib')

tree_1 = grid_tree_1.best_estimator_
tree_2 = grid_tree_2.best_estimator_
tree_3 = grid_tree_3.best_estimator_
tree_4 = grid_tree_4.best_estimator_
tree_5 = grid_tree_5.best_estimator_

tree_l = [tree_1, tree_2, tree_3, tree_4, tree_5]

# 逻辑回归模型组
grid_lr_1 = load('./models/grid_lr_1.joblib')
grid_lr_2 = load('./models/grid_lr_2.joblib')
grid_lr_3 = load('./models/grid_lr_3.joblib')
grid_lr_4 = load('./models/grid_lr_4.joblib')
grid_lr_5 = load('./models/grid_lr_5.joblib')

lr_1 = grid_lr_1.best_estimator_
lr_2 = grid_lr_2.best_estimator_
lr_3 = grid_lr_3.best_estimator_
lr_4 = grid_lr_4.best_estimator_
lr_5 = grid_lr_5.best_estimator_

lr_l = [lr_1, lr_2, lr_3, lr_4, lr_5]

In [8]:
eval1_predict_proba_RF = pd.Series(RF_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_RF = pd.Series(RF_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_RF = pd.Series(RF_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_RF = pd.Series(RF_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_RF = pd.Series(RF_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_RF = pd.concat([eval1_predict_proba_RF, 
                                   eval2_predict_proba_RF, 
                                   eval3_predict_proba_RF, 
                                   eval4_predict_proba_RF, 
                                   eval5_predict_proba_RF]).sort_index()

eval1_predict_proba_tree = pd.Series(tree_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_tree = pd.Series(tree_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_tree = pd.Series(tree_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_tree = pd.Series(tree_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_tree = pd.Series(tree_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_tree = pd.concat([eval1_predict_proba_tree, 
                                     eval2_predict_proba_tree, 
                                     eval3_predict_proba_tree, 
                                     eval4_predict_proba_tree, 
                                     eval5_predict_proba_tree]).sort_index()

eval1_predict_proba_lr = pd.Series(lr_l[0].predict_proba(X_eval1)[:, 1], index=X_eval1.index)
eval2_predict_proba_lr = pd.Series(lr_l[1].predict_proba(X_eval2)[:, 1], index=X_eval2.index)
eval3_predict_proba_lr = pd.Series(lr_l[2].predict_proba(X_eval3)[:, 1], index=X_eval3.index)
eval4_predict_proba_lr = pd.Series(lr_l[3].predict_proba(X_eval4)[:, 1], index=X_eval4.index)
eval5_predict_proba_lr = pd.Series(lr_l[4].predict_proba(X_eval5)[:, 1], index=X_eval5.index)

eval_predict_proba_lr = pd.concat([eval1_predict_proba_lr, 
                                   eval2_predict_proba_lr, 
                                   eval3_predict_proba_lr, 
                                   eval4_predict_proba_lr, 
                                   eval5_predict_proba_lr]).sort_index()

- 回归问题与分类问题融合方法差异

&emsp;&emsp;总的来说，无论是分类问题还是回归问题，模型融合的方法大类和基本融合思想并无本质区别，并且都一定程度被过拟合问题所困扰。在实际应用过程中，除了不同问题要带入不同类型模型来进行建模，还有两点区别需要注意：
- 首先是硬投票方法簇只能处理分类问题，包括硬投票、绝对多数票和排序融合法等方法，只适用于分类问题；
- 其次，对于Stacking和Blending来说，回归问题的元学习器选取主要以贝叶斯回归为主，有时也会考虑岭回归和LASSO；

当然，关于更多回归问题元学习器选取与优化的方法，会在实践过程中进行详细探讨。而对于所涉及到的新的贝叶斯回归模型，我们也将在本节对其进行介绍。

- 回归问题中模型融合效果与建模难度

&emsp;&emsp;尽管方法类似，但模型融合应用于回归问题时，往往更容易获得一个更好的结果。

&emsp;&emsp;在此前的分类问题模型融合实践过程中我们发现，对于分类问题，融合一个更好的结果其实并不容易，往往简单的通用方法会适得其反，而只有广泛尝试优化策略、并因地制宜进行调整，才往往能够获得一个更好的融合结果。但对于回归问题，可能并不存在这样的困扰。大多数回归问题在套用一个基础融合流程时就能活动一定程度的效果提升，并且很多优化方法也能起到立竿见影的效果。因此对于实践环节来说，就通过融合来提升效果这一点而言，回归问题要比分类问题更容易。

&emsp;&emsp;究其原因，其实是因为回归问题的数值预测给了第二阶段学习更大的学习空间。无论是平均法还是学习结合器法，由于数据的重复学习，过拟合问题始终是模型融合的“头号大敌”。且尽管分类问题大多数情况是围绕概率值进行连续性数值的预测和计算，但最终输出的预测结果仍然是离散变量，融合过程中更加丰富的数值多样性表现不一定能直接体现在最终的预测结果上。而回归问题则自始自终都是围绕连续性数值进行预测，哪怕融合结果有千分之一的数值提升，都能提升最终结果评估指标。此外，就建模难度而言，回归问题的建模难度往往高于分类问题，而更高的建模难度也给复杂的融合流程提供了更大的学习空间，适得一些在分类问题上容易过拟合的流程、在回归问题上反而能起到提高泛化能力的效果。

&emsp;&emsp;当然，效果更易提升并不代表回归问题整体模型融合执行流程变得更简单。实际上，对于回归问题来说，简单流程能提升效果、复杂流程更有可能提升更大。因此，面对回归问题，我们也需要反复多次尝试，训练构建多组模型融合结果来则优输出，并且如果是在测试集标签未知的情况下，也同样是需要多次线上提交来测试最终融合效果的。这一点并不会随着融合出效果的难易而发生变化。

> 在实际应用场景中，总的来看，分类问题的场景略多于回归问题。

In [9]:
test_predict_proba_RF = []

for i in range(5):
    test_predict_proba_RF.append(RF_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_RF = np.array(test_predict_proba_RF)
test_predict_proba_RF = test_predict_proba_RF.mean(0)

test_predict_proba_tree = []

for i in range(5):
    test_predict_proba_tree.append(tree_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_tree = np.array(test_predict_proba_tree)
test_predict_proba_tree = test_predict_proba_tree.mean(0)

test_predict_proba_lr = []

for i in range(5):
    test_predict_proba_lr.append(lr_l[i].predict_proba(X_test_OE)[:, 1])

test_predict_proba_lr = np.array(test_predict_proba_lr)
test_predict_proba_lr = test_predict_proba_lr.mean(0)

## <center>Ch.3.11 回归集成算法优化策略

&emsp;&emsp;在此前的内容中，我们已经完整介绍了目前模型融合中全部常用方法及其优化技巧，但之前的内容都是基于分类问题进行的技术介绍。本节开始，我们将进一步探讨这些模型融合方法在回归类问题中的应用方法，并在manual_ensemble函数库添加更多支持回归问题融合的函数。

- 数据集准备

&emsp;&emsp;接下来我们将借助一个回归数据集，来尝试进行回归类问题的模型融合。这里我们选取sklearn中的加州房价数据集:[California Housing dataset](https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html)来进行建模实验。该数据集是一个来自StatLib的统计数据集，属于统计数据集中偏难得数据集，但对于当前机器学习来说建模难度并不大，非常适合实验性质建模训练。

> 所谓统计数据集，往往指的是出于科研或调研需要，人工搜集统计的来得数据集。例如鸢尾花数据集就是一个统计数据集，出于生物科学研究，统计学家、生物学家Fisher围绕鸢尾花的不同性状特征来进行的精细化测量和记录的结果。由于统计数据集数据记录成本较高，因此数据集的特征往往是经过精心设计的，且往往都和标签具备业务上的关联性。并且，每条数据往往也是按照标准格式进行记录，相当于是清洗后的数据，这也是自动化数据采集设备出现之前人们进行数据记录的常态。

> 本节内容重点介绍如何在回归问题中进行有效的模型融合，后续案例中将会有更加复杂的回归问题建模。

&emsp;&emsp;该数据集可以直接通过如下方式从sklearn中导入：

In [10]:
from sklearn.datasets import fetch_california_housing

sklearn导入的数据对象类型是data类型，我们需要从中提取我们所需的数据特征、标签、特征名称等，并将其转化为Dataframe格式类型：

In [11]:
# 加载数据
cal_housing = fetch_california_housing()
# 提取特征及特征名称，并转化为DataFrame
X = pd.DataFrame(cal_housing.data, columns=cal_housing.feature_names)
# 提取标签
y = cal_housing.target

In [12]:
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [13]:
y

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [14]:
X.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


该数据源自上世纪90年代美国人口普查统计得到的数据，集记录了加利福尼亚地区房屋价值的中位数，其中标签（房屋价值）的单位是10万美元。其中，每条数据代表每个街区统计汇总得到的数据，各字段解释如下：

| 字段 | 解释 |
| ------ | ------ |
| MedInc | 街区收入中位数（10万） |
| HouseAge | 街区房屋年龄中位数 |
| AveRooms | 街区房屋平均房间数量 |
| AveBedrms | 街区房屋平均卧室数量 |
| Population | 街区人口数量 |
| AveOccup | 街区平均家庭成员数量 |
| Latitude | 街区纬度 |
| Longitude | 街区经度 |

能够发现数据集中全部特征均为连续特征，且多为统计指标，且由于数据集并没有任何缺失值，因此不需要进行数据清洗。另外，考虑到后续模型均为集成算法，因此也不需要对该数据集进行任何数据重编码或者分箱操作，直接带入建模即可。

> 一般来说，对于多个相邻的区域，经纬度亦可视作连续变量，具体数值上的差异代表地理位置上的相邻程度，很多指标（包括该数据集的标签：房价）都和地理位置息息相关。而其他情况，例如假设数据集中统计的是不同国家不同城市的平均房价，则由于地理位置相隔较远，经纬度本身数值并无意义，此时可以考虑将经纬度进行自然数编码，将其转化为名义型离散变量，方便后续建模。

&emsp;&emsp;接下来划分训练集和测试集：

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [16]:
X_train.shape

(16512, 8)

In [17]:
X_train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
1652,6.6134,4.0,6.560729,0.939271,1552.0,3.141700,37.93,-121.97
14652,2.3578,41.0,5.455598,1.007722,1070.0,4.131274,32.80,-117.15
3548,5.5111,16.0,5.716747,1.037905,3903.0,2.689869,34.25,-118.61
6730,8.1124,52.0,6.623188,1.019324,1153.0,2.785024,34.11,-118.14
18445,6.2957,25.0,6.627832,1.008091,2128.0,3.443366,37.25,-121.81


然后重置index：

In [18]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

In [19]:
X_train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,6.6134,4.0,6.560729,0.939271,1552.0,3.141700,37.93,-121.97
1,2.3578,41.0,5.455598,1.007722,1070.0,4.131274,32.80,-117.15
2,5.5111,16.0,5.716747,1.037905,3903.0,2.689869,34.25,-118.61
3,8.1124,52.0,6.623188,1.019324,1153.0,2.785024,34.11,-118.14
4,6.2957,25.0,6.627832,1.008091,2128.0,3.443366,37.25,-121.81


> 此外，由于数据集整体数据较为规整，且为了快速进入到模型融合环节，此处省略数据探索部分操作。

## 十二、回归问题的集成学习超参数优化策略

&emsp;&emsp;对于当前回归任务，这里考虑分别训练随机森林、极端随机树和GBDT三个模型，测试单模建模效果，并作为一级学习器带入后续的融合环节中。这三个模型属于集成学习中较强的算法，同时也是sklearn内部集成的效果最好的集成学习算法，也是算法工程人员必须掌握使用的三个算法。

&emsp;&emsp;接下来，首先围绕这三个算法进行模型训练与优化。

In [20]:
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

然后测试单模在默认超参数情况下下的预测能力，这里以MSE作为模型评估指标，模型测试结果如下：

In [51]:
# 随机森林
reg1 = RandomForestRegressor(random_state=12).fit(X_train, y_train)
print('The results of RandomForest:')
print('Train-MSE: %f, Test-MSE: %f' % (mean_squared_error(reg1.predict(X_train), y_train), 
                                       mean_squared_error(reg1.predict(X_test), y_test)))

# 极端随机树
reg2 = ExtraTreesRegressor(random_state=12).fit(X_train, y_train)
print('The results of ExtraTrees:')
print('Train-MSE: %f, Test-MSE: %f' % (mean_squared_error(reg2.predict(X_train), y_train), 
                                       mean_squared_error(reg2.predict(X_test), y_test)))

# GBDT
reg3 = GradientBoostingRegressor(random_state=12).fit(X_train, y_train)
print('The results of GBDT:')
print('Train-MSE: %f, Test-MSE: %f' % (mean_squared_error(reg3.predict(X_train), y_train), 
                                       mean_squared_error(reg3.predict(X_test), y_test)))

The results of RandomForest:
Train-MSE: 0.035168, Test-MSE: 0.250861
The results of ExtraTrees:
Train-MSE: 0.000000, Test-MSE: 0.243643
The results of GBDT:
Train-MSE: 0.260555, Test-MSE: 0.277794


能够看出，三个模型在默认参数下都取得了较好的预测结果，其中随机森林和极端随机树出现了一定的过拟合情况，有待后续通过超参数优化来解决这一问题。

&emsp;&emsp;并且，通过这组结果我们能够看出，极端随机树和随机森林模型表现较为接近，甚至对数据的拟合能力有过之而无不及。其实在大多数实际情况下，对于简单数据集，极端随机树往往会表现出非常强的预测能力，而对于复杂数据集，则整体性能会弱于随机森林，这也就是所谓的方差较小但偏差较大的模型的具体性能体现。也正是由于该特性，使得极端随机树具备了作为基础学习器进一步参与“集成”的可能性，在模型融合过程中该模型是非常值得尝试的一级学习器，而深度森林算法，则更是将其视作唯一的基础学习器。

&emsp;&emsp;接下来进一步考虑对这三个模型进行超参数优化。这里我们仍然考虑使用TPE搜索对其进行超参数优化。出于某些情况考虑，我们先计算这组模型在默认超参数情况下交叉验证均值得分情况：

In [164]:
# 随机森林
reg1 = RandomForestRegressor(random_state=12)
print('The results of RandomForest:')
print('Train-cross-mean: %f' % -(cross_val_score(reg1, X_train, y_train, scoring='neg_mean_squared_error', n_jobs=15)).mean())

# 极端随机树
reg2 = ExtraTreesRegressor(random_state=12)
print('The results of ExtraTrees:')
print('Train-cross-mean: %f' % -(cross_val_score(reg2, X_train, y_train, scoring='neg_mean_squared_error', n_jobs=15)).mean())

# GBDT
reg3 = GradientBoostingRegressor(random_state=12)
print('The results of GBDT:')
print('Train-cross-mean: %f' % -(cross_val_score(reg3, X_train, y_train, scoring='neg_mean_squared_error', n_jobs=15)).mean())

The results of RandomForest:
Train-cross-mean: 0.259973
The results of ExtraTrees:
Train-cross-mean: 0.258106
The results of GBDT:
Train-cross-mean: 0.287060


| 模型 | 训练集得分 | 交叉验证得分 | 测试集得分 |
| ------ | ------ | ------ | ------ |
| <center>随机森林 | <center>0.0351 |<center> 0.2599 | <center>0.2508 |
| <center>极端随机树 | <center>0 |<center> 0.2581 | <center>0.2436 |
| <center>GBDT | <center>0.2605 |<center> 0.2870 | <center>0.2777 |

能够发现，相比训练集得分，交叉验证得分和测试集得分更加接近。

### 1.回归随机森林超参数优化

#### 1.1 基于TPE的回归随机森林模型超参数搜索

&emsp;&emsp;首先是随机森林模型。对于回归随机森林模型的超参数优化，具体操作层面和随机森林的分类模型并无区别。尽管随机森林回归模型的criterion和分类模型的criterion可选参数不同，但在实际超参数优化过程中并不建议对其进行调整。该参数的调整对结果并无明显改善，且调整默认criterion取值也将极大程度影响模型运算速度。因此我们几乎可以创建一个和分类随机森林模型超参数搜索空间完全相同的搜索空间进行超参数优化：

In [84]:
RandomForestRegressor?

Init signature:
RandomForestRegressor(
    n_estimators=100,
    *,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)
Docstring:     
A random forest regressor.

A random forest is a meta estimator that fits a number of classifying
decision trees on various sub-samples of the dataset and uses averaging
to improve the predictive accuracy and control over-fitting.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the whole dataset is used to build
each tree.

Read more in the :ref:`User Guide <forest>`.

Parameters
----------
n_estimators : int, default=100
    The number of trees in the forest.

    .. version

&emsp;&emsp;首先我们仿造分类随机森林的超参数设置，创建超参数搜索空间如下：

In [24]:
RF_space = {'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 20, 1), 
            'min_samples_split': hp.quniform('min_samples_split', 2, 20, 1), 
            'max_depth': hp.quniform('max_depth', 2, 50, 1), 
            'max_leaf_nodes': hp.quniform('max_leaf_nodes', 50, 300, 1), 
            'n_estimators': hp.quniform('n_estimators', 50, 500, 1), 
            'max_samples': hp.uniform('max_samples', 0.2, 0.8)}

&emsp;&emsp;对于上述超参数搜索空间，有两点需要请注意，其一是超参数搜索空间创建函数选择上，这里在创建超参数搜索空间时对于连续型整数搜索空间采用了quniform的方法进行创建，通过该方法创建的搜索空间最终返回最优解将是一个空间内最优解取值，而非最优解的索引值，如此一来，相比choice创建的搜索空间（返回的是索引），使用quniform则可在定义目标函数时不再区分训练状态和测试状态不同的超参数读取流程。但这里由两点需要注意：其一，quniform默认是在连续变量之间搜索，虽然通过设置步长可以锁定在几个整数值之间搜索，但默认仍然是假定搜索空间是连续分布的，对于贝叶斯估计来说，尽管对连续分布空间的估计会更快，但连续分布中的估计方式并不适用于本质上仍然是离散取值的一组变量，因此（相比choice方法）估计的精度会下降；其二，quniform创建的搜索空间，每次读取的超参数取值都是连续型变量，需要将其转化为整型再输出到模型中，否则模型将无法读取参数。

&emsp;&emsp;其二则是关于具体超参数选择及超参数空间范围选择。根据此前介绍的决策树和随机森林基本原理可知，相比分类问题，回归问题中树的复杂度往往要高得多，因此回归随机森林中对于单独每个树的剪枝参数往往需要设置一个较大的搜索空间，例如在当前问题中，max_leaf_nodes就设置了在50到300之间，这其实就相当于给单独基础分类器的复杂度设置了较大的容忍空间。接下来定义目标函数：

In [86]:
def RF_param_objective(params, train=True):
    
    # 超参数读取
    min_samples_leaf = int(params['min_samples_leaf'])
    min_samples_split = int(params['min_samples_split'])
    max_depth = int(params['max_depth'])
    max_leaf_nodes = int(params['max_leaf_nodes'])
    n_estimators = int(params['n_estimators'])
    max_samples = params['max_samples']

    # 模型创建
    reg_RF = RandomForestRegressor(min_samples_leaf = min_samples_leaf, 
                                   min_samples_split = min_samples_split,
                                   max_depth = max_depth, 
                                   max_leaf_nodes = max_leaf_nodes, 
                                   n_estimators = n_estimators, 
                                   max_samples = max_samples)

    if train == True:
        res = -cross_val_score(reg_RF, X_train, y_train, scoring='neg_mean_squared_error', n_jobs=15).mean()
    else:
        res = reg_RF.fit(X_train, y_train)
        
    return res

同样，训练过程返回MSE，测试过程返回训练好的模型。其中neg_mean_squared_error返回的是-MSE，再加上符号最终得到MSE，然后求MSE最小值。

&emsp;&emsp;优化函数定义如下：

In [87]:
def RF_param_search(max_evals=500):
    params_best = fmin(RF_param_objective,
                       space = RF_space,
                       algo = tpe.suggest,
                       max_evals = max_evals)
    return params_best

然后开始执行超参数搜索：

In [88]:
RF_best_param = RF_param_search()

100%|███████████████████████████████████████████| 500/500 [1:05:48<00:00,  7.90s/trial, best loss: 0.28503053546834717]


查看最优超参数组：

In [89]:
RF_best_param

{'max_depth': 19.0,
 'max_leaf_nodes': 300.0,
 'max_samples': 0.774285170020038,
 'min_samples_leaf': 3.0,
 'min_samples_split': 19.0,
 'n_estimators': 425.0}

然后输出最佳超参数组在训练集上训练得到的模型：

In [90]:
RF_reg = RF_param_objective(RF_best_param, train=False)

In [91]:
RF_reg

RandomForestRegressor(max_depth=19, max_leaf_nodes=300,
                      max_samples=0.774285170020038, min_samples_leaf=3,
                      min_samples_split=19, n_estimators=425)

测试模型在训练集和测试集上表现：

In [92]:
mean_squared_error(RF_reg.predict(X_train), y_train), mean_squared_error(RF_reg.predict(X_test), y_test)

(0.1943075293502321, 0.2760535922830271)

| 训练方式 | 训练集得分 | 交叉验证得分 | 测试集得分 |
| ------ | ------ | ------ | ------ |
| 默认超参数 | <center>0.0351 |<center>0.2599 | <center>0.2508 |
| 小范围搜索 | <center>0.1943 |<center>0.2850 | <center>0.2760 |

通过上述结果，有以下几个要点需要深度探讨：

- 首先，基于交叉验证评分的搜索过程是有效的        
&emsp;&emsp;经过超参数优化，模型过拟合倾向已得到较为有效的抑制，尽管目前训练集上评分和测试集评分还是有一定差距（还是存在一定过拟合倾向），但交叉验证中验证集的平均得分和测试集得分接近，说明交叉验证得分已经能够较好的衡量模型的泛化能力，因此搜索迭代的方向也是朝着提升模型泛化能力方向进行的，上述过程是有效的；但于此同时，模型在测试集上的评分仍然较低，甚至比默认超参数情况下预测结果更差，而在搜索流程没问题的情况下，问题可能就出在超参数搜索空间的设计上；

- 其次，简单回归问题也需要较大的超参数搜索空间        
&emsp;&emsp;根据超参数搜索结果我们发现，哪怕是在一个较大的超参数搜索空间内进行搜索，部分参数的最佳取值仍然取得了（或者逼近了）搜索空间的上限（'max_leaf_nodes'和'n_estimators'），这说明对于当前数据集，超参数的搜索空间还应该更进一步放大。但进一步放大超参数搜索空间必然导致搜索时间进一步增加，且这个进一步放大的空间的边界也不易确定，关于如何确定超参数搜索空间，其实也就是很多回归问题超参数优化过程面临的最核心问题；

 - 其三，当前的过拟合风险实际上是由于超参数搜索不完整所导致的        
&emsp;&emsp;能够发现，此处为何经过超优化后的模型仍然存在一定过拟合问题，也是一个非常值得深度探讨的问题。我们知道此时模型参与搜索的部分超参数的取值是已经确定了的，但其他还有一些没有搜索的超参数，其默认取值是按照模型复杂度最高的标准进行的设置，因此这会导致模型在训练参数的过程仍然还是具备一定的过拟合风险。不过该问题并不会很大程度影响模型搜索效果——因为交叉验证能很好的得到一组和测试集较为接近的评分。因此，在以交叉验证结果作为模型泛化能力评估标准的时候，这种过拟合并不会影响超参数搜索优化过程；

> 需要注意的是，这种因为带入搜索的超参数不完全所导致的过拟合风险，主要体现在回归类模型的超参数搜索中，分类问题往往没有这种问题。并且相对简单的回归数据集问题尤其明显。

> 此外，切莫以为这里的过拟合风险是搜索空间太大导致，若裁剪搜索空间，不仅不会消除过拟合风险，反而会降低模型学习能力。这里简单进行一组测试，当缩小超参数空间时，最终模型学习能力如下：<center><img src="http://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20221030161102038.png" alt="image-20221030161102038" style="zoom:50%;" />

#### 1.2 大范围超参数搜索优化（方案A）

&emsp;&emsp;基于此，后续应该如何进行优化呢？

&emsp;&emsp;首先也是最容易想到的策略就是增加搜索的超参数，以此降低模型过拟合风险，并且同时增加各超参数搜索范围，以此提升模型泛化能力。这当然是理论上的最优方案，但实际执行难度较大，最核心的困难就在于算力不够。目前的数据集在当前搜索空间范围内迭代500次需要1小时时间，而如果增加超参数数量、并增加搜索空间，超参数优化计算所需时间将呈指数级上升。此外，对于部分模型的部分超参数甚至根本无法带入进行搜索，例如回归随机森林中的criterion参数，根据参数说明可知，当criterion取值为absolute_error时将极大程度影响计算效率、计算时间将大幅提升，这里我们可以简单进行测试：

In [33]:
start = time.time()
RandomForestRegressor().fit(X_train, y_train)
print(time.time()-start)

6.248455762863159


In [37]:
start = time.time()
RandomForestRegressor(criterion="absolute_error").fit(X_train, y_train)
print(time.time()-start)

308.26254439353943


能够发现计算时间增幅达50倍！当然这不仅仅是回归随机森林单独模型的问题，很多模型在处理回归问题时都有类似的超参数设置情况，稍后建模的回归GBDT模型更是有过之而无不及。

&emsp;&emsp;这其实也给我们启示，在算力限制情况下，我们其实根本无法带入全参数并在超大空间内进行搜索，必须要舍弃部分参数，进而兼顾计算效率和预测结果。根据长期实践经验，综合考虑当前算力情况，这里我们重新设计超参数搜索过程如下：

In [56]:
max_features_range = ["auto", "sqrt", "log2", None] + np.arange(0.1, 1., 0.1).tolist()
max_features_range

['auto',
 'sqrt',
 'log2',
 None,
 0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9]

In [55]:
RF_space = {'max_features': hp.choice('max_features', max_features_range),
            'n_estimators': hp.quniform('n_estimators', 200, 800, 1), 
            'max_samples': hp.uniform('max_samples', 0.2, 1),
            'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 50, 1), 
            'min_samples_split': hp.quniform('min_samples_split', 2, 50, 1), 
            'max_depth': hp.quniform('max_depth', 20, 120, 1), 
            'max_leaf_nodes': hp.quniform('max_leaf_nodes', 200, 600, 1)}

这里我们加入加入了'max_features'特征分配超参数，并设置了该超参数的搜索范围max_features_range，该超参数的搜索范围较为特殊，需要在一组同时包含了字符串和浮点数的列表中进行搜索，因此采用了hp.choice的方式进行搜索空间创建。此外这里也增加了基础学习器个数的搜索空间。关于超参数搜索空设置，可以按照此前课程介绍的“小步前进、快速迭代”的方式，先设置较少迭代次数、反复试出一个大概的最佳取值范围，然后再设置一个大的搜索范围进行更高精度的搜索。这里展示的是经过多次尝试后最终决定的超参数搜索范围。       
&emsp;&emsp;接下来继续定义目标函数和优化函数，这里由于增加了choice方式搜索，因此超参数的读取需要区分训练模式和测试模式：

In [70]:
def RF_param_objective(params, train=True):
    
    # 超参数读取
    min_samples_leaf = int(params['min_samples_leaf'])
    min_samples_split = int(params['min_samples_split'])
    max_depth = int(params['max_depth'])
    max_leaf_nodes = int(params['max_leaf_nodes'])
    n_estimators = int(params['n_estimators'])
    max_samples = params['max_samples']

    if train == True:
        max_features = params['max_features']
        
    else:
        max_features = max_features_range[params['max_features']]
        
    # 模型创建
    reg_RF = RandomForestRegressor(min_samples_leaf = min_samples_leaf, 
                                   min_samples_split = min_samples_split,
                                   max_depth = max_depth, 
                                   max_leaf_nodes = max_leaf_nodes, 
                                   n_estimators = n_estimators, 
                                   max_samples = max_samples,
                                   max_features = max_features,
                                   random_state=12)

    if train == True:
        res = -cross_val_score(reg_RF, X_train, y_train, scoring='neg_mean_squared_error', n_jobs=15).mean()
    else:
        res = reg_RF.fit(X_train, y_train)
        
    return res

In [71]:
def RF_param_search(max_evals=500):
    params_best = fmin(RF_param_objective,
                       space = RF_space,
                       algo = tpe.suggest,
                       max_evals = max_evals)
    return params_best

In [76]:
RF_best_param = RF_param_search(1000)

100%|██████████████████████████████████████████| 1000/1000 [2:12:03<00:00,  7.92s/trial, best loss: 0.2578642167906655]


然后测试超参数优化效果：

In [77]:
RF_reg_A = RF_param_objective(RF_best_param, train=False)

In [79]:
mean_squared_error(RF_reg_A.predict(X_train), y_train), mean_squared_error(RF_reg_A.predict(X_test), y_test)

(0.1328016085201358, 0.2500023597601782)

我们发现最终结果有了明显提升，不仅测试集准确率有了较大提升，并且训练集和测试集的评分也保持一个相对接近的水平。当然，由于当前超参数搜索仍然不完整，因此训练集上仍然还有一定的过拟合倾向。但同样，由于我们可用通过交叉验证得分评估模型泛化能力，训练集上过拟合倾向并不会对搜索过程有效性造成任何影响。

| 训练方式 | 训练集得分 | 交叉验证得分 | 测试集得分 |
| ------ | ------ | ------ | ------ |
| 默认超参数 | <center>0.0351 |<center>0.2599 | <center>0.2508 |
| 小范围搜索 | <center>0.1943 |<center>0.2850 | <center>0.2760 |
| 大范围搜索（A） | <center>0.1328 |<center>0.2578 | <center>0.2500 |

但是，这个搜索优化的过程所耗费的计算时间也达到了2个小时，需要知道的是，当前数据和特征情况还是相对简单的情况，真实企业级应用场景中（包括很多竞赛中）数据情况远比当前数据集更加复杂，考虑到可能还需要进行特征衍生、以及融合环节还需要多次训练模型，计算时间将是无法承担的。因此，必须要考虑一种更加高效的模型训练流程。

#### 1.3 更高效的搜索流程（方案B）

&emsp;&emsp;这里我们考虑另一种更高效的搜索流程：直接舍弃基础学习器的超参数搜索，只搜索集成相关超参数。这里需要注意的是，舍弃基础学习器的超参数搜索其实就等于带入基础学习器的默认超参数组，而基础学习器的默认超参数组是不会剪枝的、即默认执行最高模型复杂度的模型训练，每个基础学习器实际上是容易过拟合的，但是考虑到回归问题本身就要求更复杂的模型进行训练，且已被上述搜索过程验证，因此基础学习器选取默认超参数并不会对预测结果有较大的影响。并且除了基础学习器的超参数外、对模型泛化能力影响更大的继承超参数仍然是朝着泛化能力更强的方向进行搜索，外加舍弃了这部分超参数搜索，整体超参数搜索的速度将变得更快，这将有助于在更短的时间内快速搜索出一组更好的结果。        
&emsp;&emsp;但是需要注意，如果接下来采用方案二进行超参数搜索，当我们舍弃了更多的基础学习器的超参数的时候，最终训练集的预测结果将会有更大的过拟合风险，因此在方案二的超参数搜索时，仍然需要坚持使用交叉验证结果作为模型泛化能力的评分标准。

&emsp;&emsp;接下来我们尝试方案二的搜索流程，即直接舍弃基础学习器的超参数搜索，转而进行更加全面深入的集成参数的搜索。具体搜索流程如下：

In [43]:
RF_space = {'max_features': hp.choice('max_features', max_features_range),
            'n_estimators': hp.quniform('n_estimators', 20, 700, 1), 
            'max_samples': hp.uniform('max_samples', 0.2, 1)}

这里我们删除了'max_depth'、'max_leaf_nodes'等基础学习器的超参数，加入了'max_features'特征分配超参数，并设置了该超参数的搜索范围max_features_range，该超参数的搜索范围较为特殊，需要在一组同时包含了字符串和浮点数的列表中进行搜索，因此采用了hp.choice的方式进行搜索空间创建。并且这里略微增加了基础学习器个数的搜索空间。

&emsp;&emsp;接下来定义目标函数和优化函数。这里由于增加了choice方式搜索，因此超参数的读取需要区分训练模式和测试模式：

In [44]:
def RF_param_objective(params, train=True):
    
    # 超参数读取
    n_estimators = int(params['n_estimators'])
    max_samples = params['max_samples']

    if train == True:
        max_features = params['max_features']
        
    else:
        max_features = max_features_range[params['max_features']]
        
    # 模型创建
    reg_RF = RandomForestRegressor(n_estimators = n_estimators, 
                                   max_samples = max_samples, 
                                   max_features = max_features,
                                   random_state=12)

    if train == True:
        res = -cross_val_score(reg_RF, X_train, y_train, scoring='neg_mean_squared_error', n_jobs=15).mean()
    else:
        res = reg_RF.fit(X_train, y_train)
        
    return res

In [45]:
def RF_param_search(max_evals=500):
    params_best = fmin(RF_param_objective,
                       space = RF_space,
                       algo = tpe.suggest,
                       max_evals = max_evals)
    return params_best

然后进行超参数搜索，这里我们先基础尝试50次搜索，得到结果如下：

In [46]:
RF_best_param = RF_param_search(50)

100%|███████████████████████████████████████████████| 50/50 [08:20<00:00, 10.00s/trial, best loss: 0.24145571958697482]


In [47]:
RF_best_param

{'max_features': 2, 'max_samples': 0.9990956320729892, 'n_estimators': 619.0}

In [48]:
RF_reg_B1 = RF_param_objective(RF_best_param, train=False)

In [49]:
mean_squared_error(RF_reg_B1.predict(X_train), y_train), mean_squared_error(RF_reg_B1.predict(X_test), y_test)

(0.03217159873008787, 0.2305968021002135)

能够发现，在少量迭代时，该流程就已经能获得一个不错的预测结果，测试集结果明显提升，这也是目前得到的最好结果，并且10分钟的计算时长也是完全可以接受的。总的来说，高效搜索流程通过削减带入搜索的超参数个数，反而提升了最终模型效果。

| 训练方式 | 训练集得分 | 交叉验证得分 | 测试集得分 |
| ------ | ------ | ------ | ------ |
| 默认超参数 | <center>0.0351 |<center>0.2599 | <center>0.2508 |
| 小范围搜索 | <center>0.1943 |<center>0.2850 | <center>0.2760 |
| 大范围搜索（A） | <center>0.1328 |<center>0.2578 | <center>0.2500 |
| 高效搜索（B） | <center>0.0321 |<center>0.2414 | <center>0.2305 |

当然，此时由于诸多基础学习器的超参数未经过优化，因此在训练集上的过拟合风险加剧，但交叉验证的平均得分和测试集评分接近，说明交叉验证的平均得分仍然能够非常好的衡量模型泛化能力，说明模型搜索过程仍然有效。当然，通过和此前结果对比我们不难判断，训练集上的过拟合问题确实是由于超参数设置和搜索不完全导致的。

- 增加迭代次数尝试

&emsp;&emsp;并且，当我们判断超参数优化方向是正确的时候，甚至可以增加更多迭代次数测试最终效果。但对于当前相对简单的数据情况来说，增加迭代次数对最终效果提升影响不大，大家可以课后自行进行尝试，这里仅展示迭代500次数和1000次的计算时间和最终结果：

&emsp;&emsp;500次迭代计算时，耗时104分钟，测试集评分为0.2312

<center><img src="http://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20221030215915746.png" alt="image-20221030215915746" style="zoom:50%;" />

&emsp;&emsp;1000次迭代计算时，耗时104分钟，测试集准确率为0.2307

<center><img src="http://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20221030215849538.png" alt="image-20221030215849538" style="zoom:50%;" />

能够发现，迭代次数的增加并不能进一步提升模型效果，这也说明当前设置的50次计算是一种非常高效的获得较好结果的方式。

> 当然，TPE搜索的最佳迭代次数也和数据复杂度、模型复杂度、超参数空间设置有很大关系，并没有固定的最佳迭代次数。当然我们也可以通过设置提前停止参数来尽可能的寻定一个较为合适的迭代次数。

- 高效搜索流程可用性探讨

&emsp;&emsp;从唯结果论的角度来看，方案B无疑是最佳策略，仅用了10分钟就获得了一个比方案A的2个小时计算更好的结果，省时省力。但不管怎样，方案B训练得到的模型在训练集上仍然还是表现出了过拟合倾向，相信还是有很多小伙伴对方案B训练的模型可用性存疑。需要知道的是，尽管训练集上存在过拟合倾向，但我们对模型泛化能力的判别完全可以使用交叉验证结果，而非训练集上准确率，因此，无论是模型之间的横向比较、还是模型内部超参数搜索，训练集上准确率并不会造成任何影响。因此，该结果和整体流程仍然是可用的。

&emsp;&emsp;当然，我们这里也需要分析这四组不同的模型结果背后所代表的含义，帮助大家从实践角度理解过拟合和模型学习能力之间的辩证关系：

| 模型编号 | 训练方式 | 训练集得分 | 交叉验证得分 | 测试集得分 |
| ------ | ------ | ------ | ------ | ------ |
| <center>1| 默认超参数 | <center>0.0351 |<center>0.2599 | <center>0.2508 |
| <center>2| 小范围搜索 | <center>0.1943 |<center>0.2850 | <center>0.2760 |
| <center>2| 大范围搜索（A） | <center>0.1328 |<center>0.2578 | <center>0.2500 |
| <center>4| 高效搜索（B） | <center>0.0321 |<center>0.2414 | <center>0.2305 |

首先需要介绍一个模型超参数优化过程中的基本矛盾，那就是手段和目的不匹配的问题。即模型优化的最终目的是为了让模型提升泛化能力，即尽可能学习全局规律而抛弃局部规律，但所采用的手段——超参数搜索其实是一种限制（或者增加）模型学习能力的策略，这二者并不完全匹配。无论是局部规律还是全局规律，都有难的规律和简单的规律。而超参数搜索无论是提高模型学习能力还是降低模型学习能力，都必然会同时学习到或者损失掉一些全局规律和局部规律。因此，并非限制模型学习能力，损失掉的、没学到的规律就一定是局部规律，就一定有助于模型解决过拟合问题；反之也一样，并不是提升模型学习能力，就一定能学到全局规律，有可能也会学到局部规律，从而降低模型在测试集上的评分。这点在实践过程中屡见不鲜。相信大家在实践过程中也遇到过不少类似的情况。

&emsp;&emsp;但超参数搜索为何还是有效的呢？因为超参数搜索能够在通过调配模型学习能力，最终让模型获得一个最佳的学习能力——能最大程度同时捕获适用于训练集和测试集的规律，因而提升模型的泛化能力，也就是提升模型对测试集的预测能力。但是，从理论上来说，要做到这点必须要对模型进行全超参数的高精度搜索才行，例如上述模型3基本就是这种情况。但是全参数大范围高精度搜索其实是非常费时费力的，稍有不慎就可能得不到这个最佳的模型，反而可能会得到一个学习能力不足、泛化能力不够的模型——例如上述模型2。

&emsp;&emsp;那要如何改善这点呢？当然，与其说是改善，不如说是算力不够时“两权相害取其轻”的结果——相比通过限制搜索空间来提高搜索效率，有的时候适度的放开模型的学习能力，哪怕让模型学到了一些局部规律，只要不严重影响测试集预测结果、不影响模型整体泛化能力，其实也没有太大问题，例如模型4。对于4号模型来说，由于不对基础模型的超参数进行限制，因此其学习能力是非常强的，但由于我们对其集成超参数进行了搜索和优化，因此仍然一定程度上对其学习能力进行了调整，在有限的调整空间下，我们无法获得类似模型3中获得的严谨结果，但仍然保证了其能够有效学习全局规律，因此最终预测集上结果有所提升。但由于还有很多不受控的超参数在影响模型的学习能力，因此模型在训练集的训练过程中也学习了非常多局部规律，导致其但从训练集上来看过拟合风险偏高。但就模型泛化能力、也就是对新数据的预测能力来说，模型4和模型3并无本质区别。

&emsp;&emsp;因此，从理论上来说，模型4的结果和背后的模型训练流程，都是可用的。

- 超参数搜索中超参数选择依据

&emsp;&emsp;接下来的问题就是，既然模型3和模型4其实都是超参数不完全的搜索流程，并且二者表现出了不同的泛化能力，那要如何才能确定搜索哪几个超参数才能达到更好效果呢？

&emsp;&emsp;首先我们需要确定一点的是，从理论层面来说，超参数搜索最有效的方法一定是全参数大范围高精度搜索，如此得到的模型一定是能够最大程度学习全局规律的模型。但遗憾的是，在实际执行过程中，尤其是回归类问题，受限于当前算力条件，该方案几乎没有可执行性。因此需要考虑挑选出部分超参数进行搜索，而只要是挑选部分超参数出来进行搜索，其实就已经是不满足理论最优解的情况了，而具体要选哪些超参数出来搜索，也不存在理论最优解。因此，关于超参数的选取，其实也是需要多次尝试并且不断积累经验的。

&emsp;&emsp;总的来说，根据长期实践经验总结，集成相关超参数肯定需要进行搜索的，例如随机森林中的'max_features'、'n_estimators'和'max_samples'等，此外对于基础学习器的超参数来说，可以优先尝试max_depth、max_leaf_nodes这两个参数，该参数对决策树模型剪枝影响巨大。当然，是否要加入max_depth、max_leaf_nodes，甚至是其他基础分类器超参数进行搜索，可以通过少量次数迭代测试效果再决定。

&emsp;&emsp;例如，此处带入max_depth搜索结果如下：

In [68]:
RF_space = {'max_features': hp.choice('max_features', max_features_range),
            'n_estimators': hp.quniform('n_estimators', 200, 800, 1), 
            'max_samples': hp.uniform('max_samples', 0.2, 1),
            'max_depth': hp.quniform('max_depth', 20, 120, 1)}

def RF_param_objective(params, train=True):
    
    # 超参数读取
    max_depth = int(params['max_depth'])
    n_estimators = int(params['n_estimators'])
    max_samples = params['max_samples']

    if train == True:
        max_features = params['max_features']
        
    else:
        max_features = max_features_range[params['max_features']]
        
    # 模型创建
    reg_RF = RandomForestRegressor(max_depth = max_depth, 
                                   n_estimators = n_estimators, 
                                   max_samples = max_samples,
                                   max_features = max_features,
                                   random_state=12)

    if train == True:
        res = -cross_val_score(reg_RF, X_train, y_train, scoring='neg_mean_squared_error', n_jobs=15).mean()
    else:
        res = reg_RF.fit(X_train, y_train)
        
    return res

def RF_param_search(max_evals=500):
    params_best = fmin(RF_param_objective,
                       space = RF_space,
                       algo = tpe.suggest,
                       max_evals = max_evals)
    return params_best

RF_param_search(50)

100%|███████████████████████████████████████████████| 50/50 [09:36<00:00, 11.53s/trial, best loss: 0.24188062380676975]


{'max_depth': 95.0,
 'max_features': 7,
 'max_samples': 0.9919926208654151,
 'n_estimators': 661.0}

比不带入情况交叉验证平均得分0.2414更差，因此这里不宜带入max_depth进行搜索。接下来继续考虑带入max_leaf_nodes的情况：

In [69]:
RF_space = {'max_features': hp.choice('max_features', max_features_range),
            'n_estimators': hp.quniform('n_estimators', 200, 800, 1), 
            'max_samples': hp.uniform('max_samples', 0.2, 1),
            'max_depth': hp.quniform('max_depth', 20, 120, 1), 
            'max_leaf_nodes': hp.quniform('max_leaf_nodes', 200, 600, 1)}

def RF_param_objective(params, train=True):
    
    # 超参数读取
    max_depth = int(params['max_depth'])
    max_leaf_nodes = int(params['max_leaf_nodes'])
    n_estimators = int(params['n_estimators'])
    max_samples = params['max_samples']

    if train == True:
        max_features = params['max_features']
        
    else:
        max_features = max_features_range[params['max_features']]
        
    # 模型创建
    reg_RF = RandomForestRegressor(max_depth = max_depth, 
                                   max_leaf_nodes = max_leaf_nodes, 
                                   n_estimators = n_estimators, 
                                   max_samples = max_samples,
                                   max_features = max_features,
                                   random_state=12)

    if train == True:
        res = -cross_val_score(reg_RF, X_train, y_train, scoring='neg_mean_squared_error', n_jobs=15).mean()
    else:
        res = reg_RF.fit(X_train, y_train)
        
    return res

def RF_param_search(max_evals=500):
    params_best = fmin(RF_param_objective,
                       space = RF_space,
                       algo = tpe.suggest,
                       max_evals = max_evals)
    return params_best

RF_param_search(50)

100%|███████████████████████████████████████████████| 50/50 [05:52<00:00,  7.05s/trial, best loss: 0.26150200005559976]


{'max_depth': 86.0,
 'max_features': 7,
 'max_leaf_nodes': 579.0,
 'max_samples': 0.7548212077512358,
 'n_estimators': 464.0}

同样，带入更多超参数后，交叉验证得分反而降低，因此对于该数据集，按照模型4的训练方法即可。

- 从实践经验对理论的补位

&emsp;&emsp;其实不仅是超参数搜索，在机器学习的很多技术实践不是对理论完美复刻，而是基于实际情况，在理论的基础上、结合实际经验，寻找更优的实现方法。例如此前我们一直强调过拟合会威胁模型泛化能力，但这里发现适度放宽模型学习能力，容忍一定过拟合倾向，在借助交叉验证平均分为判别条件情况下，反而能提升模型的泛化能力，因此无比要很多时候活学活用，才能起到更好的效果。在实践过程中我们不是一定追求一个完美的流程，而是追求一个更高效更准确的最终结果。正所谓黑猫白猫、抓住老鼠就是好猫。

- 高效搜索流程优化方法

&emsp;&emsp;当然，既然是诞生于实践过程的搜索策略，往往就会有长期实践过程中逐渐摸索出来的优化流程。对于这里我们看到的基于集成参数的高效搜索流程也不例外。整体来说，由于4号模型经过优化的超参数个数较少，因此模型在不同数据集上进行训练和预测时偏差较小但方差较大（例如max_depth取值为None时，就会根据不同数据集情况训练得到不同深度的基础树模型），目前来说，较为通用的围绕高效搜索流程算法特性设计的优化方法有以下两种，这两种方法的基本思路都源于交叉训练，希望通过类似交叉训练的过程来降低输出结果方差，提升结果稳定性，进而提高模型泛化能力：

方法一：在搜索得到一组最优超参数后，通过交叉训练的方式同时输出多组测试集的预测结果，然后求均值作为最终的预测结果；      
方法二：在超参数搜索过程中，不再使用验证集的平均得分作为搜索依据，换成验证集拼接而成的预测数据和标签之间的MSE作为搜索依据。

&emsp;&emsp;很明显，这两种方法都有非常浓厚的“模型融合”的意味在里面。其实很多方法是可以同时适用于很多场景中的，只要能解决当前问题，其实就是值得尝试的方法。其实尽管课程顺序是先介绍了模型融合中的oof数据集概念和交叉训练输出平均结果等流程，但实际上这些方法大多诞生于回归问题的单模优化过程中。

&emsp;&emsp;不过需要注意的是，这些方法并不如此前介绍的超参数搜索策略效果那么明显，只能作为备选的一种策略，或许能起到优化结果的效果。建议在实际建模过程多加尝试，多训练几组结果然后择优输出。

&emsp;&emsp;首先测试方案一：这里我们选择五折交叉训练并输出最终结果：

In [70]:
RF_reg_B1

RandomForestRegressor(max_features='log2', max_samples=0.9990956320729892,
                      n_estimators=619, random_state=12)

In [156]:
RF_reg_B1.set_params(n_jobs=15)

RandomForestRegressor(max_features='log2', max_samples=0.9990956320729892,
                      n_estimators=619, n_jobs=15, random_state=12)

In [159]:
RF_test_predict = 0
RF_train_predict = 0

kf = KFold(n_splits=5, shuffle=True, random_state=12)

for train_part_index, eval_index in kf.split(X_train, y_train):
    # 在训练集上训练
    X_train_part = X_train.loc[train_part_index]
    y_train_part = y_train[train_part_index]
    RF_reg_B1.fit(X_train_part, y_train_part)
    RF_train_predict += RF_reg_B1.predict(X_train) / 5
    RF_test_predict += RF_reg_B1.predict(X_test) / 5

In [160]:
mean_squared_error(RF_train_predict, y_train)

0.05882594631689255

In [161]:
mean_squared_error(RF_test_predict, y_test)

0.2351970974351355

In [162]:
RF_reg_B1.fit(X_train, y_train)
mean_squared_error(RF_reg_B1.predict(X_train), y_train), mean_squared_error(RF_reg_B1.predict(X_test), y_test)

(0.03217159873008787, 0.23059680210021352)

然后测试方案二，方案二的测试过程较为复杂，需要修改原始搜索流程中的目标函数：

In [101]:
RF_space = {'max_features': hp.choice('max_features', max_features_range),
            'n_estimators': hp.quniform('n_estimators', 20, 700, 1), 
            'max_samples': hp.uniform('max_samples', 0.2, 1)}

In [123]:
def RF_param_objective(params, train=True):
    
    # 超参数读取
    n_estimators = int(params['n_estimators'])
    max_samples = params['max_samples']

    if train == True:
        max_features = params['max_features']
        
    else:
        max_features = max_features_range[params['max_features']]
        
    # 模型创建
    reg_RF = RandomForestRegressor(n_estimators = n_estimators, 
                                   max_samples = max_samples, 
                                   max_features = max_features,
                                   random_state=12, 
                                   n_jobs=15)

    oof_series = pd.Series(np.empty(X_train.shape[0]))
    
    if train == True:
        kf = KFold(n_splits=5, shuffle=True, random_state=12)
        for train_part_index, eval_index in kf.split(X_train, y_train):
            # 在训练集上训练
            X_train_part = X_train.loc[train_part_index]
            y_train_part = y_train[train_part_index]
            reg_RF.fit(X_train_part, y_train_part)
            X_eval_part = X_train.loc[eval_index]
            # 将验证集上预测结果拼接入oof数据集
            oof_series.loc[eval_index] = reg_RF.predict(X_eval_part)
    
        res = mean_squared_error(oof_series, y_train)
    
    else:
        res = reg_RF.fit(X_train, y_train)
        
    return res

In [118]:
def RF_param_search(max_evals=500):
    params_best = fmin(RF_param_objective,
                       space = RF_space,
                       algo = tpe.suggest,
                       max_evals = max_evals)
    return params_best

然后进行超参数搜索，仍然是进行50次搜索，得到结果如下：

In [124]:
RF_best_param = RF_param_search(50)

100%|████████████████████████████████████████████████| 50/50 [03:21<00:00,  4.04s/trial, best loss: 0.2449185928067511]


In [125]:
RF_best_param

{'max_features': 1, 'max_samples': 0.9403838253830878, 'n_estimators': 328.0}

In [126]:
RF_reg_B2 = RF_param_objective(RF_best_param, train=False)

In [127]:
mean_squared_error(RF_reg_B2.predict(X_train), y_train), mean_squared_error(RF_reg_B2.predict(X_test), y_test)

(0.037033392939370864, 0.23450777599546047)

能够发现，这两种方法在当前模型中未能发挥优化效果。最终随机森林最优预测结果如下：

| 模型 | 训练集得分 | 交叉验证得分 | 测试集得分 |
| ------ | ------ | ------ | ------ |
| <center>随机森林 | <center>0.0351 |<center> 0.2599 | <center>0.2508 |
| <center>随机森林_OPT | <center>0.0321 |<center>0.2414 | <center>0.2305 |
| <center>极端随机树 | <center>0 |<center> 0.2581 | <center>0.2436 |
| <center>GBDT | <center>0.2605 |<center> 0.2870 | <center>0.2777 |

### 2.回归极端随机树超参数优化

&emsp;&emsp;接下来是极端随机树的回归模型的超参数优化过程。

In [93]:
ExtraTreesRegressor?

Init signature:
ExtraTreesRegressor(
    n_estimators=100,
    *,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=False,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)
Docstring:     
An extra-trees regressor.

This class implements a meta estimator that fits a number of
randomized decision trees (a.k.a. extra-trees) on various sub-samples
of the dataset and uses averaging to improve the predictive accuracy
and control over-fitting.

Read more in the :ref:`User Guide <forest>`.

Parameters
----------
n_estimators : int, default=100
    The number of trees in the forest.

    .. versionchanged:: 0.22
       The default value of ``n_estimators`` changed from 10 to 100
       in 0.22.

criterion : {"squared_error", "ab

极端随机树模型参数和随机森林参数完全一致，只有criterion参数取值不同，但该参数并不是超参数优化的对象，因此极端随机树和随机森林的超参数优化流程高度类似。

&emsp;&emsp;首先是超参数空间的设置过程，对于极端随机树来说，由于其模型训练的特殊性（随机选取子节点进行criterion计算），往往树的复杂度要高于随机森林中的树，因此可以考虑稍微放大各超参数搜索空间。当然对应的需要增加迭代次数。不过到极端随机树单模型训练时间会更短，可以对冲一部分由于超参数搜索空间所带来的搜索时间的增加。此外需要注意的是，在默认情况下ExtraTreesRegressor是不包含bootstrap过程的，并且由于极端随机树的建模流程本身随机性就比较强，再进行bootstrap可能会对模型结果的稳定性造成影响，因此超参数空间中需要删除max_samples参数：

In [138]:
ET_space = {'max_depth': hp.quniform('max_depth', 2, 50, 1), 
            'n_estimators': hp.quniform('n_estimators', 20, 700, 1), 
            'max_features': hp.choice('max_features', max_features_range)}

然后定义目标函数和优化函数：

In [173]:
def ET_param_objective(params, train=True):
    
    # 超参数读取
    max_depth = int(params['max_depth'])
    n_estimators = int(params['n_estimators'])
    
    if train == True:
        max_features = params['max_features']
        
    else:
        max_features = max_features_range[params['max_features']]
    
    # 模型创建
    reg_ET = ExtraTreesRegressor(max_depth = max_depth, 
                                 n_estimators = n_estimators, 
                                 max_features = max_features, 
                                 random_state=12)

    if train == True:
        res = -cross_val_score(reg_ET, X_train, y_train, scoring='neg_mean_squared_error', n_jobs=15).mean()
    else:
        res = reg_ET.fit(X_train, y_train)
        
    return res

In [140]:
def ET_param_search(max_evals=500):
    params_best = fmin(ET_param_objective,
                       space = ET_space,
                       algo = tpe.suggest,
                       max_evals = max_evals)
    return params_best

然后进行超参数搜索优化：

In [141]:
ET_best_param = ET_param_search(50)

100%|████████████████████████████████████████████████| 50/50 [04:27<00:00,  5.35s/trial, best loss: 0.2346537351978845]


查看最优超参数组：

In [142]:
ET_best_param

{'max_depth': 39.0, 'max_features': 9, 'n_estimators': 516.0}

然后输出最佳超参数组在训练集上训练得到的模型：

In [143]:
ET_reg = ET_param_objective(ET_best_param, train=False)

In [144]:
ET_reg

ExtraTreesRegressor(max_depth=39, max_features=0.6, n_estimators=516,
                    random_state=12)

测试模型在训练集和测试集上表现：

In [145]:
mean_squared_error(ET_reg.predict(X_train), y_train), mean_squared_error(ET_reg.predict(X_test), y_test)

(1.3687030793344334e-07, 0.22259777187847052)

同样，通过超参数优化，极端随机树也获得了一组好的预测结果，整体模型性能有所提升。

&emsp;&emsp;接下来根据训练集平均得分测试两种优化策略是否有效：

In [ ]:
ET_reg.set_params(n_jobs=15)

ExtraTreesRegressor(max_depth=39, max_features=0.6, n_estimators=516, n_jobs=15,
                    random_state=12)

In [ ]:
ET_test_predict = 0
ET_train_predict = 0

kf = KFold(n_splits=5, shuffle=True, random_state=12)

for train_part_index, eval_index in kf.split(X_train, y_train):
    # 在训练集上训练
    X_train_part = X_train.loc[train_part_index]
    y_train_part = y_train[train_part_index]
    ET_reg.fit(X_train_part, y_train_part)
    ET_train_predict += ET_reg.predict(X_train) / 5
    ET_test_predict += ET_reg.predict(X_test) / 5

In [ ]:
mean_squared_error(ET_train_predict, y_train), mean_squared_error(ET_test_predict, y_test)

(0.009386272529438165, 0.22576287904950001)

In [ ]:
def ET_param_objective(params, train=True):
    
    # 超参数读取
    max_depth = int(params['max_depth'])
    n_estimators = int(params['n_estimators'])
    
    if train == True:
        max_features = params['max_features']
        
    else:
        max_features = max_features_range[params['max_features']]
    
    # 模型创建
    reg_ET = ExtraTreesRegressor(max_depth = max_depth, 
                                 n_estimators = n_estimators, 
                                 max_features = max_features, 
                                 random_state=12, 
                                 n_jobs=15)

    oof_series = pd.Series(np.empty(X_train.shape[0]))
    
    if train == True:
        kf = KFold(n_splits=5, shuffle=True, random_state=12)
        for train_part_index, eval_index in kf.split(X_train, y_train):
            # 在训练集上训练
            X_train_part = X_train.loc[train_part_index]
            y_train_part = y_train[train_part_index]
            reg_ET.fit(X_train_part, y_train_part)
            X_eval_part = X_train.loc[eval_index]
            # 将验证集上预测结果拼接入oof数据集
            oof_series.loc[eval_index] = reg_ET.predict(X_eval_part)
    
        res = mean_squared_error(oof_series, y_train)
    else:
        res = reg_ET.fit(X_train, y_train)
        
    return res

In [ ]:
ET_best_param = ET_param_search(50)

100%|███████████████████████████████████████████████| 50/50 [03:12<00:00,  3.85s/trial, best loss: 0.23321127263835334]


In [ ]:
ET_reg2 = ET_param_objective(ET_best_param2, train=False)

In [ ]:
mean_squared_error(ET_reg2.predict(X_train), y_train), mean_squared_error(ET_reg2.predict(X_test), y_test)

(6.803886411200801e-05, 0.22332734286955555)

最终最优结果如下：

| 模型 | 训练集得分 | 交叉验证得分 | 测试集得分 |
| ------ | ------ | ------ | ------ |
| <center>随机森林 | <center>0.0351 |<center> 0.2599 | <center>0.2508 |
| <center>随机森林_OPT | <center>0.0321 |<center>0.2414 | <center>0.2305 |
| <center>极端随机树 | <center>0 |<center> 0.2581 | <center>0.2436 |
| <center>极端随机树_OPT | <center>1.36e-07 |<center> 0.2346 | <center>0.2257 |
| <center>GBDT | <center>0.2605 |<center> 0.2870 | <center>0.2777 |

### 3.回归GBDT超参数优化

&emsp;&emsp;最后是GBDT的回归模型的超参数优化过程。

In [19]:
GradientBoostingRegressor?

Init signature:
GradientBoostingRegressor(
    *,
    loss='squared_error',
    learning_rate=0.1,
    n_estimators=100,
    subsample=1.0,
    criterion='friedman_mse',
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_depth=3,
    min_impurity_decrease=0.0,
    init=None,
    random_state=None,
    max_features=None,
    alpha=0.9,
    verbose=0,
    max_leaf_nodes=None,
    warm_start=False,
    validation_fraction=0.1,
    n_iter_no_change=None,
    tol=0.0001,
    ccp_alpha=0.0,
)
Docstring:     
Gradient Boosting for regression.

GB builds an additive model in a forward stage-wise fashion;
it allows for the optimization of arbitrary differentiable loss functions.
In each stage a regression tree is fit on the negative gradient of the
given loss function.

Read more in the :ref:`User Guide <gradient_boosting>`.

Parameters
----------
loss : {'squared_error', 'absolute_error', 'huber', 'quantile'},             default='squared_error'
    Loss

&emsp;&emsp;对于GBDT来说（包括其他Boosting算法也类似），最重要是的需要考虑计算时长和优化效果如何权衡的问题，很多时候GBDT的训练或超参数搜索时长并不和模型效果呈正比，并且由于无法设置n_jobs，部分参数的微调会极大程度影响运算速度，为了尽量避免无效或者低效计算，这里有几点模型训练时的参数设置建议：

&emsp;&emsp;首先基础模型的复杂度会较大程度影响模型训练速度，并且伴随n_estimators增加呈叠加效应：

In [59]:
start = time.time()
GradientBoostingRegressor().fit(X_train, y_train)
print(time.time()-start)

2.353135585784912


In [61]:
start = time.time()
GradientBoostingRegressor(max_depth=20).fit(X_train, y_train)
print(time.time()-start)

10.385526657104492


In [60]:
start = time.time()
GradientBoostingRegressor(n_estimators=200, max_depth=20).fit(X_train, y_train)
print(time.time()-start)

21.930529832839966


因此需要合理设置控制基础模型复杂度的相关参数。此外学习率也会较大程度影响计算时间，学习率越大、单模训练时长越小：

In [63]:
start = time.time()
GradientBoostingRegressor(n_estimators=200, max_depth=20, learning_rate=0.05).fit(X_train, y_train)
print(time.time()-start)

20.78476572036743


In [62]:
start = time.time()
GradientBoostingRegressor(n_estimators=200, max_depth=20, learning_rate=0.5).fit(X_train, y_train)
print(time.time()-start)

7.4038450717926025


而最关键的是，一般来说不建议调整或搜索loss和criterion两个参数。这两个参数在默认参数设置下就能起到较好效果，而若更换两个参数的取值，尤其是涉及到绝对值计算（也就是loss选择'absolute_error'或criterion选择'mae'）时，在模型较为复杂时，计算时长将呈指数级上升：

In [107]:
start = time.time()
GradientBoostingRegressor(loss='absolute_error').fit(X_train, y_train)
print(time.time()-start)

2.5370254516601562


loss选取'absolute_error'在简单模型的情况下计算时间的延长并不明显：

In [104]:
start = time.time()
GradientBoostingRegressor(n_estimators=200, max_depth=20, loss='absolute_error').fit(X_train, y_train)
print(time.time()-start)

83.54344344139099


但当模型复杂度提升后，loss选取'absolute_error'时的计算时间是默认参数'squared_error'的3-4倍，而若是在模型更加复杂的时候、或者多次建模进行超参数搜索时，loss选取'absolute_error'带来的计算时长将呈指数级上升。类似的情况还发生在criterion选取'mae'时。因此，在大多数情况下，loss和criterion可以考虑保留默认参数进行后续的超参数搜索。

In [110]:
gc.collect()

3209

&emsp;&emsp;接下来进行GBDT超参数搜索优化：

In [3]:
GBR_space = {'n_estimators': hp.quniform('n_estimators', 20, 701, 1),
             'learning_rate': hp.uniform('learning_rate', 0.02, 0.2),
             'subsample': hp.uniform('subsample', 0.1, 1.0),
             'max_depth': hp.quniform('max_depth', 2, 20, 1)}

In [150]:
def GBR_param_objective(params, train=True):
    n_estimators = int(params['n_estimators'])
    learning_rate = params['learning_rate']
    subsample = params['subsample']
    max_depth = int(params['max_depth'])
    
    reg_GBR = GradientBoostingRegressor(n_estimators = n_estimators, 
                                        learning_rate = learning_rate, 
                                        subsample = subsample, 
                                        max_depth = max_depth, 
                                        random_state=12)
    if train == True:
        res = -cross_val_score(reg_GBR, X_train, y_train, scoring='neg_mean_squared_error', n_jobs=15).mean()
    else:
        res = reg_GBR.fit(X_train, y_train)
    return res

In [148]:
def GBR_param_search(max_evals=500):
    params_best = fmin(GBR_param_objective,
                       space = GBR_space,
                       algo = tpe.suggest,
                       max_evals = max_evals)
    return params_best

In [151]:
GBR_best_param = GBR_param_search(50)

100%|███████████████████████████████████████████████| 50/50 [09:45<00:00, 11.71s/trial, best loss: 0.20911175863049003]


In [152]:
GBR_best_param

{'learning_rate': 0.07948277691156985,
 'max_depth': 7.0,
 'n_estimators': 499.0,
 'subsample': 0.9053472864274166}

然后输出最优超参数模型，并测试模型在训练集和测试集上的效果：

In [153]:
GBR_reg = GBR_param_objective(GBR_best_param, train=False)

In [154]:
GBR_reg

GradientBoostingRegressor(learning_rate=0.07948277691156985, max_depth=7,
                          n_estimators=499, random_state=12,
                          subsample=0.9053472864274166)

In [155]:
mean_squared_error(GBR_reg.predict(X_train), y_train), mean_squared_error(GBR_reg.predict(X_test), y_test)

(0.02536793820105726, 0.19804404020292288)

&emsp;&emsp;接下来根据训练集平均得分测试两种优化策略是否有效：

In [181]:
GBR_test_predict = 0
GBR_train_predict = 0

kf = KFold(n_splits=5, shuffle=True, random_state=12)

for train_part_index, eval_index in kf.split(X_train, y_train):
    # 在训练集上训练
    X_train_part = X_train.loc[train_part_index]
    y_train_part = y_train[train_part_index]
    GBR_reg.fit(X_train_part, y_train_part)
    GBR_train_predict += GBR_reg.predict(X_train) / 5
    GBR_test_predict += GBR_reg.predict(X_test) / 5

In [182]:
mean_squared_error(GBR_train_predict, y_train), mean_squared_error(GBR_test_predict, y_test)

(0.03541031209862538, 0.1916646210811945)

In [183]:
def GBR_param_objective(params, train=True):
    n_estimators = int(params['n_estimators'])
    learning_rate = params['learning_rate']
    subsample = params['subsample']
    max_depth = int(params['max_depth'])
    
    reg_GBR = GradientBoostingRegressor(n_estimators = n_estimators, 
                                        learning_rate = learning_rate, 
                                        subsample = subsample, 
                                        max_depth = max_depth, 
                                        random_state = 12)
    
    oof_series = pd.Series(np.empty(X_train.shape[0]))
    
    if train == True:
        kf = KFold(n_splits=5, shuffle=True, random_state=12)
        for train_part_index, eval_index in kf.split(X_train, y_train):
            # 在训练集上训练
            X_train_part = X_train.loc[train_part_index]
            y_train_part = y_train[train_part_index]
            reg_GBR.fit(X_train_part, y_train_part)
            X_eval_part = X_train.loc[eval_index]
            # 将验证集上预测结果拼接入oof数据集
            oof_series.loc[eval_index] = reg_GBR.predict(X_eval_part)
    
        res = mean_squared_error(oof_series, y_train)
    else:
        res = reg_GBR.fit(X_train, y_train)
    return res

In [184]:
GBR_best_param = GBR_param_search(50)

100%|███████████████████████████████████████████████| 50/50 [42:54<00:00, 51.49s/trial, best loss: 0.21289865067490696]


In [185]:
GBR_best_param

{'learning_rate': 0.09171075672824927,
 'max_depth': 5.0,
 'n_estimators': 475.0,
 'subsample': 0.7500351292461755}

In [186]:
GBR_reg2 = GBR_param_objective(GBR_best_param, train=False)

In [187]:
GBR_reg2

GradientBoostingRegressor(learning_rate=0.09171075672824927, max_depth=5,
                          n_estimators=475, random_state=12,
                          subsample=0.7500351292461755)

In [188]:
mean_squared_error(GBR_reg2.predict(X_train), y_train), mean_squared_error(GBR_reg2.predict(X_test), y_test)

(0.07978385844498767, 0.20153019448532147)

最终模型最优结果如下：

| 模型 | 训练集得分 | 交叉验证得分 | 测试集得分 |
| ------ | ------ | ------ | ------ |
| <center>随机森林 | <center>0.0351 |<center> 0.2599 | <center>0.2508 |
| <center>随机森林_OPT | <center>0.0321 |<center>0.2414 | <center>0.2305 |
| <center>极端随机树 | <center>0 |<center> 0.2581 | <center>0.2436 |
| <center>极端随机树_OPT | <center>1.36e-07 |<center> 0.2346 | <center>0.2257 |
| <center>GBDT | <center>0.2605 |<center> 0.2870 | <center>0.2777 |
| <center>GBDT_OPT | <center>0.03545 |<center> 0.2091 | <center>0.1916 |

- 模型保存

&emsp;&emsp;至此，我们就准备好了接下来进行模型融合的基础模型，这里我们将这些模型进行本地保存，方便后续调用：

In [195]:
dump(RF_reg, './models/RF_reg_B1.joblib')
dump(ET_reg, './models/ET_reg.joblib')
dump(GBR_reg, './models/GBR_reg.joblib')

['./models/GBR_reg.joblib']

In [126]:
# 调用时可使用如下语句
#RF_reg = load('./models/RF_reg.joblib')
#ET_reg = load('./models/ET_reg.joblib')
#GBR_reg = load('./models/GBR_reg.joblib')